In [1]:
%load_ext autoreload
%autoreload 2

In [75]:
import numpy as np
import pandas as pd
from zwad.ad.preprocess import *
from zwad.ad.postprocess import *

from sklearn.cluster import Birch, KMeans

In [43]:
datadir = os.path.join('..', 'data')

oid, feature = load_dataset(os.path.join(datadir, 'oid_m31.dat'), os.path.join(datadir, 'feature_m31.dat'))
# Optionally with feature names:
# oid, feature = load_dataset(os.path.join(datadir, 'oid_m31.dat'), os.path.join(datadir, 'feature_m31.dat'), os.path.join(datadir, 'feature_m31.name'))
anomalies_gmm = load_ad_table(os.path.join(datadir, 'm31_gmm.csv'))

anomaly_oid = anomalies_gmm['oid']
anomaly_feature = extract_anomaly_features(anomaly_oid, oid, feature)

In [44]:
anomaly_feature

array([[2.3800001e+00, 2.9370630e-01, 4.8951048e-02, ..., 1.0970883e+00,
        8.7504232e-01, 1.6457632e+01],
       [2.0444999e+00, 3.5802469e-01, 6.1728396e-03, ..., 1.0822589e+00,
        9.0305829e-01, 1.6626255e+01],
       [8.0999947e-01, 3.9370079e-03, 3.9370079e-03, ..., 9.9654287e-02,
        3.1402901e-01, 1.6848158e+01],
       ...,
       [2.0720000e+00, 2.2222222e-01, 4.2735044e-02, ..., 6.2597132e-01,
        7.3348981e-01, 1.6854887e+01],
       [3.2999992e-01, 6.1224490e-02, 8.1632650e-03, ..., 4.4723526e-02,
        5.1094729e-01, 1.6834017e+01],
       [1.3650036e-01, 3.5826772e-01, 3.9370079e-03, ..., 7.0678927e-02,
        8.8457823e-01, 1.5309025e+01]], dtype=float32)

In [26]:
expert_table = load_expert_table(os.path.join(datadir, 'm31_maria.csv')).reset_index(drop=True)
expert_table

,oid,tag,tag_detailed,comments,alerts
0,695211400034403,artefact,bright star,bright star,NaN
1,695211400124577,artefact,bright star,bright star,NaN
2,695211400102351,artefact,bright star,bright star,NaN
3,695211400053697,artefact,bright star,bright star,NaN
4,695211200075348,transient,red star,"MASTER transient, red star; has spectra but st...",NaN
...,...,...,...,...,...
96,695211300007276,Cepheid,NaN,NaN,NaN
97,695211400066438,artefact,spaceship,spaceship,NaN
98,695211100015190,uncat,red star,Variable; red star,NaN
99,695211200050499,artefact,bad column,bad column,NaN


In [76]:
cl = KMeans(n_clusters=5)
clustering = cl.fit_predict(scale_values(anomaly_feature))

In [79]:
table = pd.DataFrame({'oid': anomaly_oid, 'cluster': clustering}).merge(expert_table[['oid', 'tag', 'tag_detailed']])
table = table.sort_values(by='cluster', kind='mergesort')

In [80]:
table

,oid,cluster,tag,tag_detailed
5,695211400053697,0,artefact,bright star
31,695211400134715,0,artefact,M31 center
37,695211400134621,0,artefact,M31 center
2,695211200009221,1,artefact,bad column
6,695211200020939,1,artefact,bad column
7,695211200008801,1,artefact,bad column
8,695211200015230,1,artefact,bad column
13,695211200008024,1,artefact,bad column
14,695211200009296,1,artefact,bad column
15,695211200009492,1,artefact,bad column
